# Best Score

Nous avons construit ce dataset afin d'obtenir le meilleur score possible :  
1) Preprocessing pour ajouter les t0sj  
2) prédire les p0q0 à partir du dataset de soumission  
3) écrire par dessus les p0q0 lorsque l'on peut trouver algorithmiquement la valeur exacte

## 1. Preprocessing

In [ ]:
# From the course library
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/feature_engineering.py
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/visualization.py

#libraries
import feature_engineering
import visualization
import pandas as pd # For dataframes (tables)
import numpy as np # For numerical operations
import math
import matplotlib.pyplot as plt # For plots
import os # To import external python files
import sys# To import external python files
import sklearn
import missingno as msno
from pandas.plotting import scatter_matrix

--2023-07-01 13:16:29--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/feature_engineering.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2206 (2.2K) [text/plain]
Saving to: ‘feature_engineering.py’

feature_engineering 100%[===================>]   2.15K  --.-KB/s    in 0s      

2023-07-01 13:16:30 (55.1 MB/s) - ‘feature_engineering.py’ saved [2206/2206]

--2023-07-01 13:16:30--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/visualization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443.

In [ ]:
#dataset
import pandas as pd
!pip install GitPython
!git clone https://github.com/hrandrIAga/ChallengeData_SNCFTransilien.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
Cloning into 'ML_for_networks'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (195/195), done.
Receiving objects: 100% (230/230), 10.90 MiB | 4.28 MiB/s, done.
remote: Total 230 (delta 91), reused 148 (delta 20), pack-reused 0
Resolving deltas: 100% (91/91), done.


In [ ]:
df = pd.read_csv('/content/ChallengeData_SNCFTransilien/NET4550_Haga_Pierre_Adam/Datasets/Datasets_initiaux/Xsubmit.csv', sep = ';')
dataframe[['00']] = ''

In [ ]:
dataframe['date'] = pd.to_datetime(dataframe['date'])

# Extraire le jour de la semaine et l'ajouter comme colonne 'day'
dataframe['day'] = dataframe['date'].dt.day_name()

# Appliquer le One-Hot Encoding sur la colonne 'Day'
one_hot_encoded = pd.get_dummies(dataframe['day'])

one_hot_encoded = one_hot_encoded.add_prefix('Day_')

# Ajouter les colonnes encodées au DataFrame d'origine
dataframe = pd.concat([dataframe, one_hot_encoded], axis=1)

<ipython-input-5-8cd18dae60c6>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dataframe['date'] = pd.to_datetime(dataframe['date'])


In [ ]:
# Appliquer le One-Hot Encoding sur la colonne 'train'
one_hot_encoded = pd.get_dummies(dataframe['train'])

# Renommer les colonnes générées avec un préfixe 'Train_'
one_hot_encoded = one_hot_encoded.add_prefix('Train_')

# Ajouter les colonnes encodées au DataFrame d'origine
dataframe = pd.concat([dataframe, one_hot_encoded], axis=1)

In [ ]:
# Appliquer le One-Hot Encoding sur la colonne 'station'
one_hot_encoded = pd.get_dummies(dataframe['station'])
# Renommer les colonnes générées avec un préfixe 'Station_'

one_hot_encoded = one_hot_encoded.add_prefix('Station_')

# Ajouter les colonnes encodées au DataFrame d'origine
dataframe = pd.concat([dataframe, one_hot_encoded], axis=1)

In [ ]:
dataframe['hour'] = dataframe['hour'].fillna('10:00:00')

In [ ]:
hour_int = dataframe['hour']
hour_int = hour_int.str[:2].astype(int)
dataframe['hour_int'] = hour_int

In [ ]:
dataframe = dataframe.drop('way', axis  =1)

In [ ]:
dataframe = dataframe[dataframe['composition'] != 1]

In [ ]:

# Fonction pour vérifier la présence de NaN et assigner les valeurs correspondantes
def assign_filter(row):
    filter_value = ''
    filter_value += 'x' if pd.isna(row['p1q0']) else 'o'
    filter_value += 'x' if pd.isna(row['p2q0']) else 'o'
    filter_value += 'x' if pd.isna(row['p3q0']) else 'o'
    filter_value += 'x' if pd.isna(row['p0q1']) else 'o'
    filter_value += 'x' if pd.isna(row['p0q2']) else 'o'
    filter_value += 'x' if pd.isna(row['p0q3']) else 'o'
    return filter_value

# Ajouter la colonne 'filtre' au DataFrame en appliquant la fonction assign_filter sur chaque ligne
dataframe['filtre'] = dataframe.apply(assign_filter, axis=1)

### t0sj

In [ ]:
dataframe[['t0s1']] = ''
dataframe[['t0s2']] = ''
dataframe[['t0s3']] = ''

In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 1 <= row['train'] <= 11:
        if row['station'] == 'AD':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AI':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AJ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'AK':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AM':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AT':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q2']
        elif row['station'] == 'AW':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'AX':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BB':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BD':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BE':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        else:
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']



In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 12 <= row['train'] <= 21:
        if row['station'] == 'AE':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AI':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AJ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'AK':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AV':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AL':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AO':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AQ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BB':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'BC':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        else:
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']

In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 22 <= row['train'] <= 33:
        if row['station'] == 'AB':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AI':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AJ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q2']
        elif row['station'] == 'AK':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AL':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AN':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AO':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q2']
        elif row['station'] == 'AQ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AS':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BC':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'BF':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BG':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BH':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q2']
        else:
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']

In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if row['train'] == 28:
        if row['station'] == 'AE':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AI':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AJ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'AK':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AL':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AO':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AQ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BB':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'BC':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BG':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        else:
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']

In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 34 <= row['train'] <= 44:
        if row['station'] == 'AF':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AJ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AP':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AZ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BB':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        else:
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']


In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 45 <= row['train'] <= 50:
        if row['station'] == 'AA':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'AC':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'AG':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AH':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AJ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'AR':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AU':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q2']
        elif row['station'] == 'AZ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'BA':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BB':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'BI':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'BJ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q2']
        else:
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']


In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 51 <= row['train'] <= 55:
        if row['station'] == 'AC':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q2']
        elif row['station'] == 'AG':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AH':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AJ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'AU':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AV':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AY':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']
        elif row['station'] == 'AZ':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        elif row['station'] == 'BB':
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q3']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q1']
        else:
                dataframe.loc[index, 't0s1'] = dataframe.loc[index, 'p0q1']
                dataframe.loc[index, 't0s2'] = dataframe.loc[index, 'p0q2']
                dataframe.loc[index, 't0s3'] = dataframe.loc[index, 'p0q3']


In [ ]:
dataframe['val_p1q0'] = np.where(dataframe['p1q0'].isnull(), 0, 1)
dataframe['val_p2q0'] = np.where(dataframe['p2q0'].isnull(), 0, 1)
dataframe['val_p3q0'] = np.where(dataframe['p3q0'].isnull(), 0, 1)
dataframe['val_t0s1'] = np.where(dataframe['t0s1'].isnull(), 0, 1)
dataframe['val_t0s2'] = np.where(dataframe['t0s2'].isnull(), 0, 1)
dataframe['val_t0s3'] = np.where(dataframe['t0s3'].isnull(), 0, 1)

In [ ]:
dataframe.to_csv('Xtest_t0sj.csv')

## 2. Neural Network

Par la suite, nous avons télécharger le fichier Xtest_t0sj.csv, puis on a inféré à l'aide de notre modèle le plus performant  : (le réseau de neurones de3)[]

Puis on a téléchargé cette prédiction dans le fichier pred0107_nn.csv

## 3. Smart move
Enfin, on importe le fichier sur lequel on a réalisé la prédiction à l'aide du réseau de neurone du [notebook : 3_Model_FullDataset](https://colab.research.google.com/drive/16PmMoDXvzsJl9yHDuEDrXx8q0rF_82JU?usp=sharing) (pred_submission_nn.csv).  
Puis, on va chercher à l'aide des algorithmes ci dessous les bons p0q0 dans les variables historiques de l'arrêt suivant

In [ ]:
df = pd.read_csv('/content/ChallengeData_SNCFTransilien/NET4550_Haga_Pierre_Adam/Datasets/Datasets_new/pred_submission_nn.csv', sep = ',')

In [ ]:
dataframe['00'] = df['00']

In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 1 <= row['train'] <= 11:
        if row['station'] == 'AX':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BD')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'BD':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BE')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'BE':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AW')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AW':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AT')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AT':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AM')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AM':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AD')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AD':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AK')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AK':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AI')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AI':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AJ')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AJ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BB')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1


In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 12 <= row['train'] <= 21:
        if row['station'] == 'BC':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AO')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AO':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AL')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AV':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AK')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AL':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AK')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AK':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AE')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AE':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AI')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AI':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AQ')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AQ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AJ')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AJ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BB')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
                dataframe.loc[index,'t0qj'] = -1


In [ ]:
import pandas as pd


# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 22 <= row['train'] <= 33:
        if row['station'] == 'BG':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AS')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AS':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AN')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AN':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AB')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AB':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BF')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'BF':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BH')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'BH':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BC')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'BC':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AO')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AO':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AL')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AV':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AK')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AL':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AK')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AK':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AI')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AI':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AQ')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AQ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AJ')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
                dataframe.loc[index,'t0qj'] = -1


In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if row['train'] == 28:
        if row['station'] == 'BG':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BC')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'BC':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AO')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AO':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AL')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AL':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AK')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AK':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AC')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AE':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AI')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AI':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AQ')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AQ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AJ')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
                dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AJ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BB')
            if not dataframe.loc[mask].empty:
                dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
                dataframe.loc[index,'t0qj'] = -1


In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 34 <= row['train'] <= 44:
        if row['station'] == 'AZ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AP')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AP':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AF')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AF':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AJ')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AJ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BB')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
            dataframe.loc[index,'t0qj'] = -1


In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 45 <= row['train'] <= 50:
        if row['station'] == 'AR':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BA')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'BA':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BI')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'BI':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AA')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AA':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BJ')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'BJ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AU')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AU':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AC')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AC':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AG')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AG':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AH')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AH':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AZ')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AZ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AJ')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
            dataframe.loc[index,'t0qj'] = -1


In [ ]:
import pandas as pd

# Vérification des conditions et mise à jour de la colonne 00
for index, row in dataframe.iterrows():
    if 51 <= row['train'] <= 55:
        if row['station'] == 'AV':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AY')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AY':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AU')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AU':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AC')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AC':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AG')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q1'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AG':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AH')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AH':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AZ')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AZ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'AJ')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q3'].values[0]
            dataframe.loc[index,'t0qj'] = -1
        elif row['station'] == 'AJ':
            mask = (dataframe['train'] == row['train']) & (dataframe['date'] == row['date']) & (dataframe['station'] == 'BB')
            dataframe.loc[index, '00'] = dataframe.loc[mask, 'p0q2'].values[0]
            dataframe.loc[index,'t0qj'] = -1


In [ ]:
##BB

trains1=[34,35,36,37,38,39,40,41,42,43,44,51,52,53,54,55]
trains2=[1,2,3,4,5,6,7,8,9,10,11,28,38,17,19,21]
colonne=['p2q0','p2q0','p2q0','p2q0','p2q0','p2q0','p2q0','p2q0','p2q0','p2q0','p2q0','p3q0','p3q0','p3q0','p3q0','p3q0']

for index, row in dataframe.iterrows():
  if row['station'] == 'BB':
    for i in range (0,len(trains1)):
      if row['train'] == trains1[i]:
        mask = (dataframe['station'] == row['station']) & (dataframe['date'] == row['date']) & (dataframe['train'] == trains2[i])
        if not dataframe.loc[mask].empty and not pd.isnull(dataframe.loc[mask, colonne[i]].values[0]):
          dataframe.loc[index, 'p0q0'] = dataframe.loc[mask, colonne[i]].values[0]


In [ ]:
trains1=[22,23,24,25,28,29,30,31,45,46,47]
trains2=[12,28,38,17,36,13,14,26,29,30,31]
colonne=['p1q0','p2q0','p2q0','p2q0','p2q0','p2q0','p2q0','p2q0','p3q0','p3q0','p3q0']

for index, row in dataframe.iterrows():
  if row['station'] == 'AJ':
    for i in range (0,len(trains1)):
      if row['train'] == trains1[i]:
        mask = (dataframe['station'] == row['station']) & (dataframe['date'] == row['date']) & (dataframe['train'] == trains2[i])
        if not dataframe.loc[mask].empty and not pd.isnull(dataframe.loc[mask, colonne[i]].values[0]):
          dataframe.loc[index, 'p0q0'] = dataframe.loc[mask, colonne[i]].values[0]

On télécharge le fichier de prédiction pour le soummetre au challenge

In [ ]:
dataframe['00'].to_csv('submit0107B.csv')

On obtient finalement un score MAE de 0.0023, soit la 2e position du challenge comme indiqué dans les [slides](https://docs.google.com/presentation/d/1hbtUQyBCMXkWVdXpzlyux46Yxju7sGRxX_X8sbgty4A/edit?usp=drive_link)